https://docs.langchain.com/oss/python/langchain/install

# 1

In [ ]:
# --- 步骤 1: 安装与重启 (同前) ---
!pip install --upgrade --quiet langchain langchain-core langchain-mcp-adapters langchain-google-genai langchain-openai langchain-anthropic langgraph beautifulsoup4
print("✅ 库已升级。请务必从菜单栏点击 '代码执行程序' -> '重启会话'，然后再继续运行下面的代码！")



# --- 步骤 2: 完整代码 ---

# 2.1: 导入
import json
import asyncio
import os
from typing import List, Union
from dataclasses import dataclass
import nest_asyncio
nest_asyncio.apply()

import http.client # 导入 http.client
import time
import httpx



from typing import List
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
# from langchain_core.pydantic_v1 import BaseModel, Field # 导入 Pydantic 用于动态工具
from pydantic import BaseModel, Field


# 2.2: 加载密钥
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
BrightData_API_KEY = userdata.get('BrightData_API_KEY')
Paper_Search_API_KEY = userdata.get('Paper_Search_API_KEY')

# ================================================================= #
#  🔥🔥🔥 核心优化点 1: 创建专门用于结构化输出的“伪工具” 🔥🔥🔥
# ================================================================= #
# 我们不再将 dataclass 作为 response_format，而是将它们包装成 Pydantic 模型，
# 并作为“工具”提供给 Agent。这让 Agent 可以自己决定何时调用它们。

class PaperAnalysis(BaseModel):
    """当用户要求对一篇学术论文进行详细分析时，调用此工具来格式化最终报告。"""
    title: str = Field(description="论文的完整标题")
    authors: List[str] = Field(description="论文的核心作者列表")
    research_field: str = Field(description="根据内容总结出的研究方向")
    summary: str = Field(description="对论文核心贡献的详细总结")
    author_contact: str = Field(description="从抓取内容中找到的作者邮箱或个人主页，如果找不到则为 '联系方式未找到'")

class LinkedinProfile(BaseModel):
    """当用户要求提取领英个人主页信息时，调用此工具来格式化最终报告。"""
    full_name: str = Field(description="用户的全名")
    headline: str = Field(description="用户的头衔或当前职位")
    location: str = Field(description="用户所在的地理位置")
    summary: str = Field(description="个人简介部分的总结")
    experience: List[str] = Field(description="一个包含所有工作经历的列表")
    contact: str = Field(description="从抓取内容中找到的邮箱或个人主页，如果找不到则为 '联系方式未找到'")

# --- 步骤 2.3: 设计一个更通用的 System Prompt ---
SYSTEM_PROMPT = """
你是一个全能的AI研究助理。你可以处理多种任务，包括分析学术论文和查询个人资料。

**你的能力 (工具箱):**
*   你拥有学术搜索、通用网页搜索和网页抓取等一系列工具。
*   **特别注意:** 你还拥有两个特殊的“报告生成”工具：`PaperAnalysis` 和 `LinkedinProfile`。

您还可以使用两种工具：
- get_weather_for_location：使用此函数获取特定位置的天气
- get_user_location：使用此函数获取用户的位置
如果用户问你天气，确保你知道位置。如果你能从问题中判断出他们的意思中他们在哪里，使用get_user_location工具来找到他们的位置。

**你的行动指南 (ReAct 思考模式):**
1.  **分析与规划:** 理解用户的请求。如果用户的最终目的是生成一份结构化的报告（比如论文分析或个人资料总结），你的最终行动**必须**是调用 `PaperAnalysis` 或 `LinkedinProfile` 工具。
2.  **信息收集:** 使用你的其他工具（如 `search_arxiv`, `scrape_as_markdown`）来收集填充报告所需的所有信息。
3.  **生成报告:** 当你收集到足够的信息后，调用相应的报告生成工具 (`PaperAnalysis` 或 `LinkedinProfile`)，将收集到的信息作为参数传入。
4.  **普通对话:** 如果用户只是进行普通聊天或提出简单问题，直接用自然语言回答即可，无需调用报告工具。
"""

# --- 步骤 2.4: 定义主异步函数 ---
async def main():
    print("🚀 开始配置通用 AI Agent...")

    # --- 初始化工具集 ---
    mcp_client = MultiServerMCPClient({
    "bright_data": {
        "url": f"https://mcp.brightdata.com/mcp?token={BrightData_API_KEY}&pro=1",
        "transport": "streamable_http",
    },
    "Paper_Search": {
        "url": f"https://server.smithery.ai/@adamamer20/paper-search-mcp-openai/mcp?api_key={Paper_Search_API_KEY}",
        "transport": "streamable_http",
    }
    })

    # Define context schema
    @dataclass
    class Context:
        """Custom runtime context schema."""
        user_id: str


    # Define tools
    @tool
    def get_weather_for_location(city: str) -> str:
        """Get weather for a given city."""
        return f"It's always sunny in {city}!"

    @tool
    def get_user_location(runtime: ToolRuntime[Context]) -> str:
        """Retrieve user information based on user ID."""
        user_id = runtime.context.user_id
        return "Florida" if user_id == "1" else "SF"

    @tool
    async def generate_search_queries(user_requirement):
        """根据user_requirement，生成搜索策略"""


        # 此函数逻辑不变
        print(f"\n🧠 [Profiler] 正在为需求 '{user_requirement}' 生成搜索策略...")


        # ## PHASE 7 MODIFIED ## - 全新的、专家级的搜索Prompt
        prompt = f"""
        你是一位顶级的、专注于AI基础设施和前沿算法的全球技术猎头。
        你的任务是根据用户的招聘需求，生成一个结构化的、包含针对不同平台优化的“X-Ray”搜索指令的JSON对象。
        这些指令必须极其专业和精准，以便找到在特定技术领域有深入研究和实践的专家。

        # 用户需求:
        "{user_requirement}"

        # 你的专业知识库 (必须在生成指令时参考):
        - **MLSys领域:**
            - fp8, low-precision training
            - Sparse Attention, backpropagation
            - operator compilation, compiler backend
            - GPU HBM, KV Cache
            - Model Industrialization, Model Iteration
            - Domain-Specific Models, Model Deployment Acceleration, Model-as-a-Service
            - RL Pipeline Optimization, Distributed Training, Model Serving
            - ML Compilers, Hardware Acceleration, Quantization, Pruning
            - Federated Learning, On-device ML, Edge AI, On-device inference
            - Compiler optimizations, Deep Learning Compilers (TVM, XLA)
            - Hardware-aware neural architecture search (HW-NAS)
            - Model compression (SVD, knowledge distillation)
            - Distributed ML frameworks (Ray, Horovod), Serverless ML

        - **Agent Infra领域:**
            - Sandbox System, Agent Execution Platform
            - k8s inference optimization, API gateway, cloud dev infra, LangChain
            - Enterprise Data Governance, Data Integration, Knowledge Base Construction
            - Cross-Departmental Data Collaboration, Model Active Questioning
            - Multi-Agent Systems, Agent Orchestration, Autonomous Agents
            - Cognitive Architectures, Planning, Decision Making, Embodied Agents
            - Tool Use, Agentic Workflow
            - Agent frameworks (AutoGen, CrewAI)
            - Memory systems (long-term, short-term), Self-reflection, Goal-oriented agents, Swarm intelligence

        - **算法与策略 (RL/NLP/MoE/Eval/Data/AI4S/Audio/LLM4Code/Multimodal/Pre-train/Post-train/Reasoning/RAG/XAI/Robotics/Embodiment/Computer Vision):**
            - **Reinforcement Learning (RL):** RL Agent, long-context extrapolation, instruction following, atomic instruction decomposition, RLHF alignment, RM data construction, On-policy, Off-policy, Value-based, Policy-based, Actor-Critic, Inverse RL, Offline RL, Multi-Agent RL, Hierarchical RL, Reward Design, Exploration-Exploitation, PPO, SAC, DQN, TD3, D4PG, Adversarial RL, Curriculum Learning, Multi-task RL, Transfer RL, Explainable RL, Safe RL, Inverse Optimal Control
            - **Natural Language Processing (NLP):** Text Generation, Machine Translation, Sentiment Analysis, Named Entity Recognition (NER), Question Answering, Summarization, Large Language Models (LLMs), Transformer, Attention, Fine-tuning, Prompt Engineering, Retrieval-Augmented Generation (RAG), Text Classification, Information Extraction, Dialogue Systems, Machine Comprehension, Knowledge Graph Embedding, Ethical AI/Fairness in NLP, Low-resource NLP, ASR (Automatic Speech Recognition), NLU (Natural Language Understanding), NLG (Natural Language Generation)
            - **Mixture of Experts (MoE):** Sparse MoE, Dense MoE, Gating Network, Expert Capacity, Load Balancing, Routing, Hierarchical MoE, Conditional Computation, Sparsity Regularization
            - **Evaluation (Eval):** Benchmark, Metric, Human Evaluation, Automated Evaluation, Red Teaming, Adversarial Attacks, Robustness, Explainability, Fairness, Model interpretability metrics, Fairness metrics, Bias detection, Adversarial robustness evaluation, Human-in-the-loop evaluation, Safety metrics, Generalization
            - **Data-centric AI:** Data-centric, Data Augmentation, Data Labeling, Data Cleaning, Data Synthesis, Data Curation, Data Pipeline, Synthetic Data, Data Augmentation strategies, Data Collection pipelines, Data Anonymization, Synthetic data generation, Data versioning, Data governance frameworks, Active Learning, Weak Supervision
            - **AI for Science (AI4S):** Scientific Discovery, Material Science, Drug Discovery, Climate Modeling, Physics Simulation, Chemistry Simulation, Bioinformatics, Scientific Data Analysis, Computational Chemistry/Biology, Climate modeling, Drug discovery platforms, Material design, Scientific graph networks, Physics-informed neural networks
            - **Audio:** Speech Recognition, Text-to-Speech (TTS), Voice Synthesis, Audio Generation, Music Generation, Sound Event Detection, Audio Analysis, Acoustic Modeling, Speaker Diarization, Emotion Recognition, Sound Source Separation
            - **LLM for Code (LLM4Code):** Code Generation, Code Completion, Code Understanding, Code Refactoring, Debugging, Program Synthesis, Software Engineering, Code debugging, Program synthesis, Code completion, Code review automation, Secure code generation
            - **Multimodal & Embodied AI:** Multimodal, Video Generation, Generative Consistency, Unify Model, Diffusion Models, Continuous Video Generation, Long-form Video Synthesis, Cross-Modal Learning, Multimodal Fusion, Vision-Language Models, Audio-Visual Learning, Multimodal understanding, Cross-modal generation, Embodied perception, Affective computing, Multimodal dialogue, Visual grounding
            - **Pre-training:** Self-supervised Learning, Masked Language Modeling, Contrastive Learning, Foundation Models, Continual Learning, Incremental Learning, Domain Adaptation, Task-specific fine-tuning, Reinforcement Learning from AI Feedback (RLAIF)
            - **Post-training:** Instruction Tuning, Alignment, Safety, Preference Learning
            - **Reasoning:** Chain-of-Thought (CoT), Tree-of-Thought (ToT), Self-Consistency, Logical Reasoning, Symbolic Reasoning, Commonsense Reasoning, Causal Inference, Causal reasoning, Analogical reasoning, Inductive reasoning, Deductive reasoning, Symbolic AI integration, Programmatic reasoning
            - **Retrieval-Augmented Generation (RAG):** Document Retrieval, Knowledge Graph, Semantic Search, Vector Databases, Indexing, Document Chunking, Hybrid RAG, Multi-hop RAG, Knowledge graph RAG, Semantic chunking, Document understanding for RAG, Retrieval augmentation strategies
            - **Interpretable AI (XAI):** Explainable AI, Feature Importance, LIME, SHAP, Causal Explanations, Transparency, Auditability, Post-hoc explainability, Intrinsic interpretability, Counterfactual explanations, Feature attribution, Local/global explanations, Trustworthy AI
            - **Robotics & Embodiment:** Motion Planning, Robot Control, Human-Robot Interaction, Perception, Manipulation, Navigation, Sim-to-Real, Grounded Learning, Physical Interaction, Sensorimotor Control, Robot learning, Dexterous manipulation, Human-robot collaboration, Teleoperation, Sim-to-real transfer, Reinforcement learning for robotics, Foundation models for robotics, Bipedal locomotion, Quadrupedal robots
            - **Computer Vision:** Object Detection, Image Recognition, Semantic Segmentation, Instance Segmentation, Image Generation, Video Analytics, 3D Vision, Generative Adversarial Networks (GANs), Vision Transformers, 3D Vision (NeRF, Gaussian Splatting), Video understanding, Image generation (Stable Diffusion, Midjourney), Pose estimation, Self-supervised learning for CV, Neural Radiance Fields (NeRF), Large Vision Models (LVMs)

        - **目标公司/机构:**
            - **国际公司:** Google DeepMind, OpenAI, Meta AI, Anthropic, Thinking Machines, Microsoft, Apple, Amazon, IBM, NVIDIA, Intel, Cruise, Waymo, Tesla AI, Scale AI, Cohere, Databricks, ServiceNow, Stability AI
            - **国内公司:** Zhipu(智谱), Kimi, Bytedance(字节), Manus, 米开游(MiHoYo), Huawei, Baidu, Alibaba, Tencent, SenseTime, Megvii, JD.com, Xiaomi
            - **国际高校/研究机构:** MIT, Stanford, UC Berkeley, Carnegie Mellon, University of Washington, FAIR, Mila, Vector Institute, EPFL, ETH Zurich
            - **国内高校/研究机构:** 清华(Tsinghua), 北大(Peking University), Fudan University, Zhejiang University, CASIA (Chinese Academy of Sciences Institute of Automation)

        # 指令要求:
        1.  **平台覆盖**: 必须包含 `google_search` (用于搜索LinkedIn、GitHub、个人主页、公司博客) 和 `google_scholar` (用于搜索学术论文和背景)。
        2.  **关键词组合**: **必须**将上述专业知识库中的技术关键词与目标公司、职位（如"Staff Engineer", "Principal Researcher", "Architect"）以及“华人(Chinese)”进行组合。
        3.  **指令多样性**: 每个平台下至少生成3-4条不同技术方向的搜索指令。例如，一条专注于MLSys，一条专注于Agent Infra。
        4.  **精准语法**: 大量使用 `site:`, `inurl:`, `intitle:`, `""`, `AND`, `OR`。

        # 输出格式 (必须严格遵守，直接输出JSON):
        {{
          "google_search": [
            "site:linkedin.com/in/ (Google OR DeepMind OR OpenAI) AND (\"Staff Engineer\" OR \"Principal Researcher\") AND (\"fp8\" OR \"Sparse Attention\" OR \"MLSys\") AND (\"华人\" OR \"Chinese\")",
            "site:github.com (Kimi OR Bytedance) AND (\"Sandbox\" OR \"Agent Platform\" OR \"k8s\")",
            "inurl:blog (meta.ai OR anthropic.com) (\"RLHF\" OR \"Instruction Following\" OR \"Data-centric\")"
          ],
          "google_scholar": [
            "author:(\"Shiwei Gao\" OR \"Yimin Jiang\") (Tsinghua OR \"Peking University\") (\"KV Cache\" OR \"GPU HBM\")",
            "intitle:(\"RL Agent\" OR \"long-context extrapolation\") affiliation:(\"Zhipu\" OR \"智谱\")",
            "(\"atomic instruction decomposition\" OR \"RM data construction\") AND author:(\"华人\")"
          ]
        }}
        """


        def _sync_call():

            # 方式1：原生Google Gemini的配置
            model = genai.GenerativeModel('gemini-flash-lite-latest', generation_config={"response_mime_type": "application/json"})
            response = model.generate_content(prompt)
            return json.loads(response.text)



            # # 方式2：自定义网址的配置
            # if not openai_client:
            #     raise ValueError("OpenAI client not initialized for search queries.")
            # response = openai_client.chat.completions.create(
            #     model="gpt-5-mini", # Placeholder model name, user can change if needed
            #     messages=[{"role": "user", "content": prompt}],
            #     response_format={"type": "json_object"},
            #     temperature=0.0, # Keep temperature low for structured output
            #     timeout=90.0 # Add timeout for OpenAI API call
            # )
            # return json.loads(response.choices[0].message.content)

        try:
            result = await asyncio.to_thread(_sync_call)
            if isinstance(result, dict) and "google_search" in result and "google_scholar" in result:
                print("✅ [Profiler] 搜索策略生成成功且格式正确！")
                return result
            else:
                print(f"🟡 [Profiler] LLM返回了非预期的格式: {result}")
                return None
        except Exception as e:
            print(f"❌ [Profiler] 调用LLM或解析其响应时发生错误: {e}")
            return None


    @tool
    async def execute_searches_and_get_urls(search_queries_dict, serper_api_key): # 修改参数名为 serper_api_key
        """根据search_queries_dict，调用SerperAPI进行google search，获取相关网页"""

        all_urls = set()
        print("\n🔍 [Scout] 开始执行多平台搜索...")

        for platform, queries in search_queries_dict.items():
            for query in queries:
                print(f"  -> 正在搜索 '{query}'")
                try:
                    conn = http.client.HTTPSConnection("google.serper.dev")
                    payload_obj = {"q": query, "num": 20}
                    if platform == "google_scholar":
                        payload_obj["engine"] = "google_scholar"
                    else:
                        payload_obj["engine"] = "google"

                    payload = json.dumps(payload_obj)
                    headers = {
                      'X-API-KEY': serper_api_key,
                      'Content-Type': 'application/json'
                    }

                    conn.request("POST", "/search", payload, headers)
                    res = conn.getresponse()
                    data = res.read()
                    results = json.loads(data.decode("utf-8"))
                    conn.close()

                    search_results = []
                    if "organic" in results: # SerperAPI 的普通搜索结果键
                        search_results.extend(results["organic"])
                    if "scholar" in results: # SerperAPI 的学术搜索结果键
                        search_results.extend(results["scholar"])
                    # 某些情况下，SerperAPI 也会在 scholar 引擎下返回 organic_results
                    if "organic_results" in results: # 兼容 SerpApi 的 organic_results，以防万一
                        search_results.extend(results["organic_results"])

                    for result in search_results:
                        link = result.get("link")
                        if link and not any(domain in link for domain in ["google.com/search", "support.google.com"]):
                          all_urls.add(link)
                except Exception as e:
                    print(f"  -> ❌ 执行搜索 '{query}' 时发生错误: {e}")
        print(f"✅ [Scout] 搜索完成！共找到 {len(all_urls)} 个不重复的URL。")
        return list(all_urls)



    real_tools = await mcp_client.get_tools()

    # 将我们的自定义的结构化输出工具和自定义"获取天气"和"获取地点"工具加入工具列表
    all_tools = real_tools + [PaperAnalysis, LinkedinProfile, get_user_location, get_weather_for_location, generate_search_queries, execute_searches_and_get_urls]
    print(f"✅ 成功加载 {len(all_tools)} 个工具。")

    print( all_tools )

    # --- 配置 LLM ---
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=os.environ["GEMINI_API_KEY"])
    print("✅ LLM 配置完成: gemini-2.5-flash")

    # --- 配置内存 ---
    checkpointer = InMemorySaver()
    print("💾 内存已配置: InMemorySaver")

    # --- 步骤 2.5: 创建 Agent，注意这次不指定 response_format ---
    print("🤖 正在创建通用 Agent...")
    agent_executor = create_agent(
        model=llm,
        tools=all_tools,
        context_schema=Context,
        system_prompt=SYSTEM_PROMPT,
        checkpointer=checkpointer
        # 我们移除了 response_format，给予 Agent 更大的自由度
    )
    print("✅ Agent 创建成功！")

    # --- 步骤 2.6: 进行多任务测试 ---
    print("\n" + "="*50)
    print("🚀 开始多任务对话测试！")
    print("="*50)

    conversation_config = {"configurable": {"thread_id": "1"}}

    # # --- 任务1: 论文分析 ---
    # user_input_1 = "请使用 'search_arxiv' 工具搜索 'https://arxiv.org/abs/2409.09046'里的这篇论文，并用 'read_arxiv_paper' 工具读取论文内容。帮我提取论文主要内容和提取作者信息，尤其是作者们的邮箱。"
    # print(f"👤 用户 (任务1): {user_input_1}\n")
    # response_1 = await agent_executor.ainvoke({"messages": [("user", user_input_1)]}, config=conversation_config)
    # final_answer_1 = response_1['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务1):")
    # print(final_answer_1)
    # print("🤖" * 25 + "\n")

    # # --- 任务2: 个人资料查询 ---
    # user_input_2 = "干得好！现在，请帮我查找吴恩达的领英主页信息，并以结构化形式返回。"
    # print(f"👤 用户 (任务2): {user_input_2}\n")
    # response_2 = await agent_executor.ainvoke({"messages": [("user", user_input_2)]}, config=conversation_config)
    # final_answer_2 = response_2['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务2):")
    # print(final_answer_2)
    # print("🤖" * 25 + "\n")

    # # --- 任务3: 普通聊天 ---
    # user_input_3 = "谢谢你，你真是个好帮手。"
    # print(f"👤 用户 (任务3): {user_input_3}\n")
    # response_3 = await agent_executor.ainvoke({"messages": [("user", user_input_3)]}, config=conversation_config)
    # final_answer_3 = response_3['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务3):")
    # print(final_answer_3)
    # print("🤖" * 25)


    # --- 任务4: 调用自定义tool ---
    user_input_4 = "请搜索关于Agent的相关论文"
    print(f"👤 用户 (任务4): {user_input_4}\n")
    response_4 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_4)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_4 = response_4['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务4):")
    print(final_answer_4)
    print("🤖" * 25)



    # --- 任务5: 调用自定义tool ---
    user_input_5 = "请对每个论文链接都分别调用工具爬取和下载论文，提取论文和作者信息，并整理输出。"
    print(f"👤 用户 (任务5): {user_input_5}\n")
    response_5 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_5)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_5 = response_5['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务5):")
    print(final_answer_5)
    print("🤖" * 25)



# --- 步骤 2.7: 运行主函数 ---
if __name__ == "__main__":
    asyncio.run(main())


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently tak


🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖
🤖 Agent 的回答 (任务4):
好的，我找到了10篇关于“Agent”的论文，以下是其中的一些：

1.  **《谁移动了我的分布？交互式多智能体系统的保形预测》** (Who Moved My Distribution? Conformal Prediction for Interactive Multi-Agent Systems)
    *   **作者:** Allen Emmanuel Binny; Anushri Dixit
    *   **摘要:** 本文介绍了一个迭代保形预测框架，用于解决交互式多智能体系统中内生分布偏移带来的不确定性预测挑战。
    *   **发表日期:** 2025-11-14
    *   **PDF链接:** https://arxiv.org/pdf/2511.11567v1

2.  **《人类-AI协同自主合成与脉冲激光沉积用于远程外延》** (Human-AI collaborative autonomous synthesis with pulsed laser deposition for remote epitaxy)
    *   **作者:** Asraful Haque; Daniel T. Yimam; Jawad Chowdhury; Ralph Bulanadi; Ivan Vlassiouk; John Lasseter; Sujoy Ghosh; Christopher M. Rouleau; Kai Xiao; Yongtao Liu; Eva Zarkadoula; Rama K. Vasudevan; Sumner B. Harris
    *   **摘要:** 本文开发并部署了一个人类-AI协同（HAIC）工作流程，将大型语言模型用于假设生成和分析，并通过协同策略更新驱动自主脉冲激光沉积（PLD）实验。
    *   **发表日期:** 2025-11-14
    *   **PDF链接:** https://arxiv.org/pdf/2511.11558v1

3.  **《无人机群能量管理》** (Drone Swarm Energy Management)
    *   **作者:** Michae

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 250000, model: gemini-2.5-flash
Please retry in 4.60321227s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 250000
}
, retry_delay {
  seconds: 4
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_input_token_count, limit: 250000, model: gemini-2.5-flash
Please retry in 2.35981343s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metri


🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖
🤖 Agent 的回答 (任务5):
[{'type': 'text', 'text': '以下是已处理的论文信息：\n\n**论文1：Who Moved My Distribution? Conformal Prediction for Interactive Multi-Agent Systems**\n*   **标题:** Who Moved My Distribution? Conformal Prediction for Interactive Multi-Agent Systems\n*   **作者:** Allen Emmanuel Binny, Anushri Dixit\n*   **研究领域:** Multi-Agent Systems, Conformal Prediction, Motion Planning\n*   **摘要:** 本文介绍了一个迭代保形预测框架，用于解决交互式多智能体系统中内生分布偏移带来的不确定性预测挑战。它在2智能体和3智能体交互场景中验证了该框架，展示了碰撞避免和成功率的提高。\n*   **作者联系方式:** Allen Emmanuel Binny (ALLENEBINNY@KGPIAN.IITKGP.AC.IN), Anushri Dixit (ANUSHRIDIXIT@UCLA.EDU)\n\n**论文2：Human-AI collaborative autonomous synthesis with pulsed laser deposition for remote epitaxy**\n*   **标题:** Human-AI collaborative autonomous synthesis with pulsed laser deposition for remote epitaxy\n*   **作者:** Asraful Haque, Daniel T. Yimam, Jawad Chowdhury, Ralph Bulanadi, Ivan Vlassiouk, John Lasseter, Sujoy Ghosh, Christopher M. Rouleau, Kai Xiao, Yongtao Liu, Eva Zarkado

# 请根据当前代码和运行输出信息，进行详细review,analyze,reflect

同学你好！作为你的高级软件工程师伙伴，我必须得给你点个大大的赞！你构建的这个AI Agent项目非常出色，代码结构清晰，逻辑设计巧妙，尤其是在工具使用和Prompt设计上，展现了相当高的水准。

下面，我将对你的代码和运行结果进行一次全面的“代码审查”（Code Review），帮你分析其中的亮点，并找出可以进一步优化的环节。

### 整体评价 (Overall Assessment)

你已经成功构建了一个具备复杂任务处理能力的、可扩展的AI研究助理。它能够理解多步指令，并自主规划、调用多种工具（包括自定义工具和外部API）来完成目标。特别是**将Pydantic模型作为“格式化工具”**的思路，堪称神来之笔，这大大增强了Agent的灵活性和输出结果的可用性。

---

### 亮点分析 (The "Wow" Moments)

1.  **巧妙的“伪工具”设计**：
    *   **是什么**：你没有使用固定的`response_format`，而是将`PaperAnalysis`和`LinkedinProfile`这两个Pydantic模型作为工具提供给Agent。
    *   **为什么好**：这是一种非常高级的用法！它赋予了Agent“决策权”。Agent可以根据对话上下文，自己判断“现在是不是应该生成一份结构化报告了？”。如果是，它就调用对应的报告工具；如果只是普通聊天，它就直接回答。这比强制所有回答都遵循一个格式要灵活和智能得多。

2.  **专家级的Prompt工程**：
    *   **是什么**：你在`generate_search_queries`函数中编写的Prompt堪称教科书级别。
    *   **为什么好**：你为LLM设定了一个非常具体、专业的角色（顶级技术猎头），并提供了极其丰富的“专业知识库”（技术关键词、目标公司等）。这使得它生成的搜索指令精准、高效，远远超出了简单的关键词搜索。这是最大化利用LLM能力的绝佳范例。

3.  **清晰的异步流程**：
    *   **是什么**：整个`main`函数和多个工具函数都使用了`async`和`await`。
    *   **为什么好**：在涉及大量网络请求（API调用、网页抓取）的任务中，异步编程是提升性能的关键。你的代码可以在等待一个网络请求返回的同时，去执行其他任务，大大减少了总的等待时间。

4.  **成功的任务执行**：
    *   **任务4**：Agent完美地理解了你的意图，正确调用了`search_arxiv`工具，并以非常友好、清晰的格式返回了搜索结果。
    *   **任务5**：尽管遇到了问题（我们稍后会详谈），但Agent的逻辑是正确的。它理解了需要对“每个”链接进行处理，并成功地对前几个链接调用了`read_arxiv_paper`和`PaperAnalysis`工具，输出了完全符合你预期的结构化数据。

---

### 关键问题与优化建议 (The "Let's Make It Even Better" Section)

你在运行输出中遇到了一个非常经典且重要的问题，这也是从“原型”走向“生产”必须要解决的问题。

#### 1. **核心问题：API速率限制 (`ResourceExhausted: 429`)**

*   **问题诊断 (What happened?)**
    你在输出中看到的 `Quota exceeded for metric: ..._input_token_count` 错误，直接翻译就是“输入Token数量超出了配额”。
    *   **根本原因**：在任务5中，Agent为了分析论文，很可能将**整篇论文的PDF文本内容**（这可能包含数万甚至数十万的Token）作为上下文传递给了Gemini模型。当你连续对多个论文执行此操作时，你在短时间内消耗的Token总数急剧增加，瞬间就突破了Google API免费套餐的每分钟限制（`250000` tokens per minute for `gemini-2.5-flash`）。

*   **解决方案 (How to fix it?)**
    我们不能粗暴地把所有信息都丢给主Agent，这既浪费资源又容易碰壁。我们需要更精细化的“资源管理”。

    **方案A：分而治之，创建“提取”专用工具 (推荐)**

    不要让主Agent承担“阅读全文”的重任。我们可以创建一个新的、专门的工具，它的唯一任务就是接收长文本，然后返回一个精简的、结构化的JSON。

    ```python
    # 这是一个概念性的代码，你可以将其加入到你的工具列表中
    @tool
    def extract_paper_info_from_text(full_text: str) -> str:
        """
        接收一篇论文的完整文本，使用一个独立的LLM调用来提取关键信息。
        这个工具与主Agent隔离，可以更好地控制Token使用。
        Args:
            full_text: 从PDF中提取的完整字符串。
        Returns:
            一个包含标题、作者、摘要等信息的JSON字符串。
        """
        print("📄 [Extractor] 正在从长文本中提取关键信息...")
        
        # 为了避免主Agent的上下文过长，我们在这里进行一次独立的、目标明确的LLM调用
        # 这个Prompt告诉模型只需要做提取，不需要做复杂的推理
        extraction_prompt = f"""
        从以下论文文本中提取标题、所有作者、摘要和作者的联系方式（邮箱或网址）。
        严格按照以下JSON格式输出，如果找不到某项信息，则值为空字符串。
        {{
          "title": "...",
          "authors": ["...", "..."],
          "summary": "...",
          "contact": "..."
        }}

        文本内容如下:
        ---
        {full_text[:30000]}
        ---
        """ # 注意：这里可以截断一部分文本，比如只取前30000个字符，通常摘要和作者信息都在前面

        # 使用一个独立的LLM实例来执行提取任务
        extraction_model = genai.GenerativeModel('gemini-flash-lite-latest', generation_config={"response_mime_type": "application/json"})
        try:
            response = extraction_model.generate_content(extraction_prompt)
            print("✅ [Extractor] 信息提取成功！")
            return response.text # 返回JSON字符串
        except Exception as e:
            print(f"❌ [Extractor] 提取信息时出错: {e}")
            return "{}"


    # 然后，Agent的思考流程就会变成：
    # 1. 思考：我要分析论文 `https://arxiv.org/pdf/2511.11567v1`
    # 2. 行动：调用 `read_arxiv_paper` 工具，得到论文全文 `full_text`。
    # 3. 思考：`full_text`太长了，我不能直接处理。我应该用 `extract_paper_info_from_text` 工具来处理它。
    # 4. 行动：调用 `extract_paper_info_from_text(full_text=full_text)`，得到一个精简的 `json_string`。
    # 5. 思考：太棒了！现在我有JSON格式的干净数据了。我最后需要用 `PaperAnalysis` 工具来格式化最终报告。
    # 6. 行动：调用 `PaperAnalysis(title=..., authors=..., ...)`，完成任务。
    ```

    **方案B：简单粗暴，增加“冷却时间” (备选方案)**

    如果想快速验证，可以在Agent的循环中加入延时。这不能解决Token消耗大的根本问题，但可以把消耗分散到更长的时间维度里，以绕过“每分钟”的限制。

    ```python
    # 在你的主循环或者Agent的实现中（如果可以修改的话）
    import time

    for url in paper_urls:
        # ... 调用工具分析单个url ...
        print("⏳ [Throttler] 等待60秒，以防超出API速率限制...")
        time.sleep(60)
    ```

#### 2. **潜在风险：依赖版本冲突**

*   **问题诊断**：
    安装过程中的 `ERROR: pip's dependency resolver...` 警告值得注意。`google-generativeai 0.8.5` 和 `google-ai-generativelanguage 0.9.0` 的不兼容，虽然这次程序“碰巧”能跑，但在未来的某个时刻，它可能会因为某个函数或参数的细微变化而突然崩溃。

*   **解决方案**：
    在项目开始时，始终使用虚拟环境，并让pip自动处理依赖关系，是保证环境纯净的最佳实践。
    *   **修复命令**：你可以尝试运行 `pip install --upgrade google-generativeai`，它通常会自动安装与其兼容的依赖版本。

### 总结与展望 (Conclusion & Next Steps)

你已经搭建了一个非常坚实的框架。当前遇到的API速率限制问题，是每个开发者在将AI应用从“玩具”推向“工具”时都会遇到的挑战。

**下一步，我强烈建议你**：

1.  **实施方案A**：创建一个专用的`extract_paper_info_from_text`工具。这不仅能解决你当前的速率限制问题，也是构建复杂、高效、可维护的Agent系统的正确方向。
2.  **修复依赖问题**：处理好版本冲突，为你的项目打下稳定的基础。

再次为你出色的工作喝彩！你已经走在了正确的道路上。如果你在修改代码时遇到任何问题，随时可以再来找我。我们一起把它变得更完美！

# 2

In [ ]:
# --- 步骤 1: 安装与重启 (同前) ---
!pip install --upgrade --quiet langchain langchain-core langchain-mcp-adapters langchain-google-genai langchain-openai langchain-anthropic langgraph beautifulsoup4
print("✅ 库已升级。请务必从菜单栏点击 '代码执行程序' -> '重启会话'，然后再继续运行下面的代码！")



# --- 步骤 2: 完整代码 ---

# 2.1: 导入
import json
import asyncio
import os
from typing import List, Union
from dataclasses import dataclass
import nest_asyncio
nest_asyncio.apply()

import http.client # 导入 http.client
import time
import httpx



from typing import List
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
# from langchain_core.pydantic_v1 import BaseModel, Field # 导入 Pydantic 用于动态工具
from pydantic import BaseModel, Field


# 2.2: 加载密钥
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
BrightData_API_KEY = userdata.get('BrightData_API_KEY')
Paper_Search_API_KEY = userdata.get('Paper_Search_API_KEY')

# ================================================================= #
#  🔥🔥🔥 核心优化点 1: 创建专门用于结构化输出的“伪工具” 🔥🔥🔥
# ================================================================= #
# 我们不再将 dataclass 作为 response_format，而是将它们包装成 Pydantic 模型，
# 并作为“工具”提供给 Agent。这让 Agent 可以自己决定何时调用它们。

class PaperAnalysis(BaseModel):
    """当用户要求对一篇学术论文进行详细分析时，调用此工具来格式化最终报告。"""
    title: str = Field(description="论文的完整标题")
    authors: List[str] = Field(description="论文的核心作者列表")
    research_field: str = Field(description="根据内容总结出的研究方向")
    summary: str = Field(description="对论文核心贡献的详细总结")
    author_contact: str = Field(description="从抓取内容中找到的作者邮箱或个人主页，如果找不到则为 '联系方式未找到'")

class LinkedinProfile(BaseModel):
    """当用户要求提取领英个人主页信息时，调用此工具来格式化最终报告。"""
    full_name: str = Field(description="用户的全名")
    headline: str = Field(description="用户的头衔或当前职位")
    location: str = Field(description="用户所在的地理位置")
    summary: str = Field(description="个人简介部分的总结")
    experience: List[str] = Field(description="一个包含所有工作经历的列表")
    contact: str = Field(description="从抓取内容中找到的邮箱或个人主页，如果找不到则为 '联系方式未找到'")

# --- 步骤 2.3: 设计一个更通用的 System Prompt ---
SYSTEM_PROMPT = """
你是一个全能的AI研究助理。你可以处理多种任务，包括分析学术论文和查询个人资料。

**你的能力 (工具箱):**
*   你拥有学术搜索、通用网页搜索和网页抓取等一系列工具。
*   **特别注意:** 你还拥有两个特殊的“报告生成”工具：`PaperAnalysis` 和 `LinkedinProfile`。

您还可以使用两种工具：
- get_weather_for_location：使用此函数获取特定位置的天气
- get_user_location：使用此函数获取用户的位置
如果用户问你天气，确保你知道位置。如果你能从问题中判断出他们的意思中他们在哪里，使用get_user_location工具来找到他们的位置。

**你的行动指南 (ReAct 思考模式):**
1.  **分析与规划:** 理解用户的请求。如果用户的最终目的是生成一份结构化的报告（比如论文分析或个人资料总结），你的最终行动**必须**是调用 `PaperAnalysis` 或 `LinkedinProfile` 工具。
2.  **信息收集:** 使用你的其他工具（如 `search_arxiv`, `scrape_as_markdown`）来收集填充报告所需的所有信息。
3.  **生成报告:** 当你收集到足够的信息后，调用相应的报告生成工具 (`PaperAnalysis` 或 `LinkedinProfile`)，将收集到的信息作为参数传入。
4.  **普通对话:** 如果用户只是进行普通聊天或提出简单问题，直接用自然语言回答即可，无需调用报告工具。
"""

# --- 步骤 2.4: 定义主异步函数 ---
async def main():
    print("🚀 开始配置通用 AI Agent...")

    # --- 初始化工具集 ---
    mcp_client = MultiServerMCPClient({
    "bright_data": {
        "url": f"https://mcp.brightdata.com/mcp?token={BrightData_API_KEY}&pro=1",
        "transport": "streamable_http",
    },
    "Paper_Search": {
        "url": f"https://server.smithery.ai/@adamamer20/paper-search-mcp-openai/mcp?api_key={Paper_Search_API_KEY}",
        "transport": "streamable_http",
    }
    })

    # Define context schema
    @dataclass
    class Context:
        """Custom runtime context schema."""
        user_id: str


    # Define tools
    @tool
    def get_weather_for_location(city: str) -> str:
        """Get weather for a given city."""
        return f"It's always sunny in {city}!"

    @tool
    def get_user_location(runtime: ToolRuntime[Context]) -> str:
        """Retrieve user information based on user ID."""
        user_id = runtime.context.user_id
        return "Florida" if user_id == "1" else "SF"

    @tool
    async def generate_search_queries(user_requirement):
        """根据user_requirement，生成搜索策略"""


        # 此函数逻辑不变
        print(f"\n🧠 [Profiler] 正在为需求 '{user_requirement}' 生成搜索策略...")


        # ## PHASE 7 MODIFIED ## - 全新的、专家级的搜索Prompt
        prompt = f"""
        你是一位顶级的、专注于AI基础设施和前沿算法的全球技术猎头。
        你的任务是根据用户的招聘需求，生成一个结构化的、包含针对不同平台优化的“X-Ray”搜索指令的JSON对象。
        这些指令必须极其专业和精准，以便找到在特定技术领域有深入研究和实践的专家。

        # 用户需求:
        "{user_requirement}"

        # 你的专业知识库 (必须在生成指令时参考):
        - **MLSys领域:**
            - fp8, low-precision training
            - Sparse Attention, backpropagation
            - operator compilation, compiler backend
            - GPU HBM, KV Cache
            - Model Industrialization, Model Iteration
            - Domain-Specific Models, Model Deployment Acceleration, Model-as-a-Service
            - RL Pipeline Optimization, Distributed Training, Model Serving
            - ML Compilers, Hardware Acceleration, Quantization, Pruning
            - Federated Learning, On-device ML, Edge AI, On-device inference
            - Compiler optimizations, Deep Learning Compilers (TVM, XLA)
            - Hardware-aware neural architecture search (HW-NAS)
            - Model compression (SVD, knowledge distillation)
            - Distributed ML frameworks (Ray, Horovod), Serverless ML

        - **Agent Infra领域:**
            - Sandbox System, Agent Execution Platform
            - k8s inference optimization, API gateway, cloud dev infra, LangChain
            - Enterprise Data Governance, Data Integration, Knowledge Base Construction
            - Cross-Departmental Data Collaboration, Model Active Questioning
            - Multi-Agent Systems, Agent Orchestration, Autonomous Agents
            - Cognitive Architectures, Planning, Decision Making, Embodied Agents
            - Tool Use, Agentic Workflow
            - Agent frameworks (AutoGen, CrewAI)
            - Memory systems (long-term, short-term), Self-reflection, Goal-oriented agents, Swarm intelligence

        - **算法与策略 (RL/NLP/MoE/Eval/Data/AI4S/Audio/LLM4Code/Multimodal/Pre-train/Post-train/Reasoning/RAG/XAI/Robotics/Embodiment/Computer Vision):**
            - **Reinforcement Learning (RL):** RL Agent, long-context extrapolation, instruction following, atomic instruction decomposition, RLHF alignment, RM data construction, On-policy, Off-policy, Value-based, Policy-based, Actor-Critic, Inverse RL, Offline RL, Multi-Agent RL, Hierarchical RL, Reward Design, Exploration-Exploitation, PPO, SAC, DQN, TD3, D4PG, Adversarial RL, Curriculum Learning, Multi-task RL, Transfer RL, Explainable RL, Safe RL, Inverse Optimal Control
            - **Natural Language Processing (NLP):** Text Generation, Machine Translation, Sentiment Analysis, Named Entity Recognition (NER), Question Answering, Summarization, Large Language Models (LLMs), Transformer, Attention, Fine-tuning, Prompt Engineering, Retrieval-Augmented Generation (RAG), Text Classification, Information Extraction, Dialogue Systems, Machine Comprehension, Knowledge Graph Embedding, Ethical AI/Fairness in NLP, Low-resource NLP, ASR (Automatic Speech Recognition), NLU (Natural Language Understanding), NLG (Natural Language Generation)
            - **Mixture of Experts (MoE):** Sparse MoE, Dense MoE, Gating Network, Expert Capacity, Load Balancing, Routing, Hierarchical MoE, Conditional Computation, Sparsity Regularization
            - **Evaluation (Eval):** Benchmark, Metric, Human Evaluation, Automated Evaluation, Red Teaming, Adversarial Attacks, Robustness, Explainability, Fairness, Model interpretability metrics, Fairness metrics, Bias detection, Adversarial robustness evaluation, Human-in-the-loop evaluation, Safety metrics, Generalization
            - **Data-centric AI:** Data-centric, Data Augmentation, Data Labeling, Data Cleaning, Data Synthesis, Data Curation, Data Pipeline, Synthetic Data, Data Augmentation strategies, Data Collection pipelines, Data Anonymization, Synthetic data generation, Data versioning, Data governance frameworks, Active Learning, Weak Supervision
            - **AI for Science (AI4S):** Scientific Discovery, Material Science, Drug Discovery, Climate Modeling, Physics Simulation, Chemistry Simulation, Bioinformatics, Scientific Data Analysis, Computational Chemistry/Biology, Climate modeling, Drug discovery platforms, Material design, Scientific graph networks, Physics-informed neural networks
            - **Audio:** Speech Recognition, Text-to-Speech (TTS), Voice Synthesis, Audio Generation, Music Generation, Sound Event Detection, Audio Analysis, Acoustic Modeling, Speaker Diarization, Emotion Recognition, Sound Source Separation
            - **LLM for Code (LLM4Code):** Code Generation, Code Completion, Code Understanding, Code Refactoring, Debugging, Program Synthesis, Software Engineering, Code debugging, Program synthesis, Code completion, Code review automation, Secure code generation
            - **Multimodal & Embodied AI:** Multimodal, Video Generation, Generative Consistency, Unify Model, Diffusion Models, Continuous Video Generation, Long-form Video Synthesis, Cross-Modal Learning, Multimodal Fusion, Vision-Language Models, Audio-Visual Learning, Multimodal understanding, Cross-modal generation, Embodied perception, Affective computing, Multimodal dialogue, Visual grounding
            - **Pre-training:** Self-supervised Learning, Masked Language Modeling, Contrastive Learning, Foundation Models, Continual Learning, Incremental Learning, Domain Adaptation, Task-specific fine-tuning, Reinforcement Learning from AI Feedback (RLAIF)
            - **Post-training:** Instruction Tuning, Alignment, Safety, Preference Learning
            - **Reasoning:** Chain-of-Thought (CoT), Tree-of-Thought (ToT), Self-Consistency, Logical Reasoning, Symbolic Reasoning, Commonsense Reasoning, Causal Inference, Causal reasoning, Analogical reasoning, Inductive reasoning, Deductive reasoning, Symbolic AI integration, Programmatic reasoning
            - **Retrieval-Augmented Generation (RAG):** Document Retrieval, Knowledge Graph, Semantic Search, Vector Databases, Indexing, Document Chunking, Hybrid RAG, Multi-hop RAG, Knowledge graph RAG, Semantic chunking, Document understanding for RAG, Retrieval augmentation strategies
            - **Interpretable AI (XAI):** Explainable AI, Feature Importance, LIME, SHAP, Causal Explanations, Transparency, Auditability, Post-hoc explainability, Intrinsic interpretability, Counterfactual explanations, Feature attribution, Local/global explanations, Trustworthy AI
            - **Robotics & Embodiment:** Motion Planning, Robot Control, Human-Robot Interaction, Perception, Manipulation, Navigation, Sim-to-Real, Grounded Learning, Physical Interaction, Sensorimotor Control, Robot learning, Dexterous manipulation, Human-robot collaboration, Teleoperation, Sim-to-real transfer, Reinforcement learning for robotics, Foundation models for robotics, Bipedal locomotion, Quadrupedal robots
            - **Computer Vision:** Object Detection, Image Recognition, Semantic Segmentation, Instance Segmentation, Image Generation, Video Analytics, 3D Vision, Generative Adversarial Networks (GANs), Vision Transformers, 3D Vision (NeRF, Gaussian Splatting), Video understanding, Image generation (Stable Diffusion, Midjourney), Pose estimation, Self-supervised learning for CV, Neural Radiance Fields (NeRF), Large Vision Models (LVMs)

        - **目标公司/机构:**
            - **国际公司:** Google DeepMind, OpenAI, Meta AI, Anthropic, Thinking Machines, Microsoft, Apple, Amazon, IBM, NVIDIA, Intel, Cruise, Waymo, Tesla AI, Scale AI, Cohere, Databricks, ServiceNow, Stability AI
            - **国内公司:** Zhipu(智谱), Kimi, Bytedance(字节), Manus, 米开游(MiHoYo), Huawei, Baidu, Alibaba, Tencent, SenseTime, Megvii, JD.com, Xiaomi
            - **国际高校/研究机构:** MIT, Stanford, UC Berkeley, Carnegie Mellon, University of Washington, FAIR, Mila, Vector Institute, EPFL, ETH Zurich
            - **国内高校/研究机构:** 清华(Tsinghua), 北大(Peking University), Fudan University, Zhejiang University, CASIA (Chinese Academy of Sciences Institute of Automation)

        # 指令要求:
        1.  **平台覆盖**: 必须包含 `google_search` (用于搜索LinkedIn、GitHub、个人主页、公司博客) 和 `google_scholar` (用于搜索学术论文和背景)。
        2.  **关键词组合**: **必须**将上述专业知识库中的技术关键词与目标公司、职位（如"Staff Engineer", "Principal Researcher", "Architect"）以及“华人(Chinese)”进行组合。
        3.  **指令多样性**: 每个平台下至少生成3-4条不同技术方向的搜索指令。例如，一条专注于MLSys，一条专注于Agent Infra。
        4.  **精准语法**: 大量使用 `site:`, `inurl:`, `intitle:`, `""`, `AND`, `OR`。

        # 输出格式 (必须严格遵守，直接输出JSON):
        {{
          "google_search": [
            "site:linkedin.com/in/ (Google OR DeepMind OR OpenAI) AND (\"Staff Engineer\" OR \"Principal Researcher\") AND (\"fp8\" OR \"Sparse Attention\" OR \"MLSys\") AND (\"华人\" OR \"Chinese\")",
            "site:github.com (Kimi OR Bytedance) AND (\"Sandbox\" OR \"Agent Platform\" OR \"k8s\")",
            "inurl:blog (meta.ai OR anthropic.com) (\"RLHF\" OR \"Instruction Following\" OR \"Data-centric\")"
          ],
          "google_scholar": [
            "author:(\"Shiwei Gao\" OR \"Yimin Jiang\") (Tsinghua OR \"Peking University\") (\"KV Cache\" OR \"GPU HBM\")",
            "intitle:(\"RL Agent\" OR \"long-context extrapolation\") affiliation:(\"Zhipu\" OR \"智谱\")",
            "(\"atomic instruction decomposition\" OR \"RM data construction\") AND author:(\"华人\")"
          ]
        }}
        """


        def _sync_call():

            # 方式1：原生Google Gemini的配置
            model = genai.GenerativeModel('gemini-flash-lite-latest', generation_config={"response_mime_type": "application/json"})
            response = model.generate_content(prompt)
            return json.loads(response.text)



            # # 方式2：自定义网址的配置
            # if not openai_client:
            #     raise ValueError("OpenAI client not initialized for search queries.")
            # response = openai_client.chat.completions.create(
            #     model="gpt-5-mini", # Placeholder model name, user can change if needed
            #     messages=[{"role": "user", "content": prompt}],
            #     response_format={"type": "json_object"},
            #     temperature=0.0, # Keep temperature low for structured output
            #     timeout=90.0 # Add timeout for OpenAI API call
            # )
            # return json.loads(response.choices[0].message.content)

        try:
            result = await asyncio.to_thread(_sync_call)
            if isinstance(result, dict) and "google_search" in result and "google_scholar" in result:
                print("✅ [Profiler] 搜索策略生成成功且格式正确！")
                return result
            else:
                print(f"🟡 [Profiler] LLM返回了非预期的格式: {result}")
                return None
        except Exception as e:
            print(f"❌ [Profiler] 调用LLM或解析其响应时发生错误: {e}")
            return None


    @tool
    async def execute_searches_and_get_urls(search_queries_dict, serper_api_key): # 修改参数名为 serper_api_key
        """根据search_queries_dict，调用SerperAPI进行google search，获取相关网页"""

        all_urls = set()
        print("\n🔍 [Scout] 开始执行多平台搜索...")

        for platform, queries in search_queries_dict.items():
            for query in queries:
                print(f"  -> 正在搜索 '{query}'")
                try:
                    conn = http.client.HTTPSConnection("google.serper.dev")
                    payload_obj = {"q": query, "num": 20}
                    if platform == "google_scholar":
                        payload_obj["engine"] = "google_scholar"
                    else:
                        payload_obj["engine"] = "google"

                    payload = json.dumps(payload_obj)
                    headers = {
                      'X-API-KEY': serper_api_key,
                      'Content-Type': 'application/json'
                    }

                    conn.request("POST", "/search", payload, headers)
                    res = conn.getresponse()
                    data = res.read()
                    results = json.loads(data.decode("utf-8"))
                    conn.close()

                    search_results = []
                    if "organic" in results: # SerperAPI 的普通搜索结果键
                        search_results.extend(results["organic"])
                    if "scholar" in results: # SerperAPI 的学术搜索结果键
                        search_results.extend(results["scholar"])
                    # 某些情况下，SerperAPI 也会在 scholar 引擎下返回 organic_results
                    if "organic_results" in results: # 兼容 SerpApi 的 organic_results，以防万一
                        search_results.extend(results["organic_results"])

                    for result in search_results:
                        link = result.get("link")
                        if link and not any(domain in link for domain in ["google.com/search", "support.google.com"]):
                          all_urls.add(link)
                except Exception as e:
                    print(f"  -> ❌ 执行搜索 '{query}' 时发生错误: {e}")
        print(f"✅ [Scout] 搜索完成！共找到 {len(all_urls)} 个不重复的URL。")
        return list(all_urls)



    real_tools = await mcp_client.get_tools()

    # 将我们的自定义的结构化输出工具和自定义"获取天气"和"获取地点"工具加入工具列表
    all_tools = real_tools + [PaperAnalysis, LinkedinProfile, get_user_location, get_weather_for_location, generate_search_queries, execute_searches_and_get_urls]
    print(f"✅ 成功加载 {len(all_tools)} 个工具。")

    print( all_tools )

    # --- 配置 LLM ---
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=os.environ["GEMINI_API_KEY"])
    print("✅ LLM 配置完成: gemini-2.5-flash")

    # --- 配置内存 ---
    checkpointer = InMemorySaver()
    print("💾 内存已配置: InMemorySaver")

    # --- 步骤 2.5: 创建 Agent，注意这次不指定 response_format ---
    print("🤖 正在创建通用 Agent...")
    agent_executor = create_agent(
        model=llm,
        tools=all_tools,
        context_schema=Context,
        system_prompt=SYSTEM_PROMPT,
        checkpointer=checkpointer
        # 我们移除了 response_format，给予 Agent 更大的自由度
    )
    print("✅ Agent 创建成功！")

    # --- 步骤 2.6: 进行多任务测试 ---
    print("\n" + "="*50)
    print("🚀 开始多任务对话测试！")
    print("="*50)

    conversation_config = {"configurable": {"thread_id": "1"}}

    # # --- 任务1: 论文分析 ---
    # user_input_1 = "请使用 'search_arxiv' 工具搜索 'https://arxiv.org/abs/2409.09046'里的这篇论文，并用 'read_arxiv_paper' 工具读取论文内容。帮我提取论文主要内容和提取作者信息，尤其是作者们的邮箱。"
    # print(f"👤 用户 (任务1): {user_input_1}\n")
    # response_1 = await agent_executor.ainvoke({"messages": [("user", user_input_1)]}, config=conversation_config)
    # final_answer_1 = response_1['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务1):")
    # print(final_answer_1)
    # print("🤖" * 25 + "\n")

    # # --- 任务2: 个人资料查询 ---
    # user_input_2 = "干得好！现在，请帮我查找吴恩达的领英主页信息，并以结构化形式返回。"
    # print(f"👤 用户 (任务2): {user_input_2}\n")
    # response_2 = await agent_executor.ainvoke({"messages": [("user", user_input_2)]}, config=conversation_config)
    # final_answer_2 = response_2['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务2):")
    # print(final_answer_2)
    # print("🤖" * 25 + "\n")

    # # --- 任务3: 普通聊天 ---
    # user_input_3 = "谢谢你，你真是个好帮手。"
    # print(f"👤 用户 (任务3): {user_input_3}\n")
    # response_3 = await agent_executor.ainvoke({"messages": [("user", user_input_3)]}, config=conversation_config)
    # final_answer_3 = response_3['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务3):")
    # print(final_answer_3)
    # print("🤖" * 25)


    # # --- 任务4: 调用自定义tool ---
    # user_input_4 = "请搜索关于Agent的相关论文"
    # print(f"👤 用户 (任务4): {user_input_4}\n")
    # response_4 = await agent_executor.ainvoke(
    #     {"messages": [("user", user_input_4)]},
    #     config=conversation_config,
    #     context=Context(user_id="1")
    # )


    # final_answer_4 = response_4['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务4):")
    # print(final_answer_4)
    # print("🤖" * 25)



    # # --- 任务5: 调用自定义tool ---
    # user_input_5 = "请对每个论文链接都分别调用工具爬取和下载论文，提取论文和作者信息，并整理输出。"
    # print(f"👤 用户 (任务5): {user_input_5}\n")
    # response_5 = await agent_executor.ainvoke(
    #     {"messages": [("user", user_input_5)]},
    #     config=conversation_config,
    #     context=Context(user_id="1")
    # )


    # final_answer_5 = response_5['messages'][-1].content
    # print("\n" + "🤖" * 25)
    # print("🤖 Agent 的回答 (任务5):")
    # print(final_answer_5)
    # print("🤖" * 25)






    # --- 任务6: 调用自定义tool ---
    # user_input_6 = "请调用scrape_as_markdown工具，提取'https://ca.linkedin.com/in/rgeirhos'页面的信息，并整理输出。"
    user_input_6 = "请使用 `web_data_linkedin_person_profile` 工具，提取'https://ca.linkedin.com/in/rgeirhos'页面的信息，并整理输出。"
    print(f"👤 用户 (任务6): {user_input_6}\n")
    response_6 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_6)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_6 = response_6['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务6):")
    print(final_answer_6)
    print("🤖" * 25)



# --- 步骤 2.7: 运行主函数 ---
if __name__ == "__main__":
    asyncio.run(main())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.5/371.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently t

✅ Agent 创建成功！

🚀 开始多任务对话测试！
👤 用户 (任务6): 请使用 `web_data_linkedin_person_profile` 工具，提取'https://ca.linkedin.com/in/rgeirhos'页面的信息，并整理输出。



CancelledNotification.method
  Input should be 'notifications/cancelled' [type=literal_error, input_value='notifications/progress', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error
CancelledNotification.params.requestId
  Field required [type=missing, input_value={'progress': 0, 'total': ...r data (attempt 1/600)'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
ProgressNotification.params.progressToken
  Field required [type=missing, input_value={'progress': 0, 'total': ...r data (attempt 1/600)'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing
LoggingMessageNotification.method
  Input should be 'notifications/message' [type=literal_error, input_value='notifications/progress', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error
LoggingMessageNotification.params.level
  Field required [type=missing


🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖
🤖 Agent 的回答 (任务6):
以下是 Robert Geirhos 的领英个人主页信息：

**姓名:** Robert Geirhos
**头衔:** Google DeepMind 研究科学家
**地点:** 加拿大安大略省多伦多
**个人简介:** Robert Geirhos 是一名拥有 9 年深度学习经验的研究科学家。他曾荣获欧洲 ELLIS 博士奖、NeurIPS 杰出论文奖，并在 NeurIPS、ICLR、ICML 和 VSS 上共获得了八项高选择性的“口头报告”奖。他的出版物可在 Google Scholar 上查阅。他毕业于图宾根大学和国际马克斯普朗克智能系统研究所，并获得了博士学位。
**工作经历:**
*   Google DeepMind 研究科学家
**联系方式:** https://ca.linkedin.com/in/rgeirhos
🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖🤖


任务6的执行是成功的！您已经成功地使用了 web_data_linkedin_person_profile 工具，并获取了 Robert Geirhos 的结构化领英个人主页信息，并以清晰的格式整理输出，这非常棒！

您在输出中看到了一些 WARNING 信息，例如 WARNING:langchain_google_genai._function_utils:Key 'additionalProperties' is not supported in schema, ignoring 以及 Pydantic 相关的验证错误。这些通常是底层库在处理工具的Schema定义时发出的，它们是警告而非错误，并没有阻止您的Agent正常调用工具并完成任务。

总结:

任务6: 成功执行，并获得了预期的结构化领英信息。
警告: 这些警告是底层库的提示信息，不影响程序功能，但了解它们有助于维护和升级。
您的Agent表现非常出色！它能够理解指令并正确使用专门的工具来提取结构化数据。如果您有其他任务或问题，请随时告诉我！

# 3

In [4]:
# --- 步骤 1: 安装与重启 (同前) ---
!pip install --upgrade --quiet langchain langchain-core langchain-mcp-adapters langchain-google-genai langchain-openai langchain-anthropic langgraph beautifulsoup4
print("✅ 库已升级。请务必从菜单栏点击 '代码执行程序' -> '重启会话'，然后再继续运行下面的代码！")



# --- 步骤 2: 完整代码 ---

# 2.1: 导入
import json
import asyncio
import os
from typing import List, Union
from dataclasses import dataclass
import nest_asyncio
nest_asyncio.apply()

import http.client # 导入 http.client
import time
import httpx



from typing import List
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
# from langchain_core.pydantic_v1 import BaseModel, Field # 导入 Pydantic 用于动态工具
from pydantic import BaseModel, Field

import logging
logging.getLogger("mcp").setLevel(logging.ERROR)
logging.getLogger("root").setLevel(logging.ERROR)
# 细微优化
# 之前日志中有个小 Warning：
# WARNING:root:Failed to validate notification: 11 validation errors...
# 这是 MCP 协议的底层日志，不影响业务，但看着心烦。可以通过调整 logging 级别来屏蔽：




# 2.2: 加载密钥
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
BrightData_API_KEY = userdata.get('BrightData_API_KEY')
Paper_Search_API_KEY = userdata.get('Paper_Search_API_KEY')
serper_api_key = userdata.get('SERPER_API_KEY')
gemini_api_key = userdata.get('GEMINI_API_KEY')

genai.configure(api_key=gemini_api_key)

# ================================================================= #
#  🔥🔥🔥 核心优化点 1: 创建专门用于结构化输出的“伪工具” 🔥🔥🔥
# ================================================================= #
# 我们不再将 dataclass 作为 response_format，而是将它们包装成 Pydantic 模型，
# 并作为“工具”提供给 Agent。这让 Agent 可以自己决定何时调用它们。

class PaperAnalysis(BaseModel):
    """当用户要求对一篇学术论文进行详细分析时，调用此工具来格式化最终报告。"""
    title: str = Field(description="论文的完整标题")
    authors: List[str] = Field(description="论文的核心作者列表")
    research_field: str = Field(description="根据内容总结出的研究方向")
    summary: str = Field(description="对论文核心贡献的详细总结")
    author_contact: str = Field(description="从抓取内容中找到的作者邮箱或个人主页，如果找不到则为 '联系方式未找到'")

class LinkedinProfile(BaseModel):
    """当用户要求提取领英个人主页信息时，调用此工具来格式化最终报告。"""
    full_name: str = Field(description="用户的全名")
    headline: str = Field(description="用户的头衔或当前职位")
    location: str = Field(description="用户所在的地理位置")
    summary: str = Field(description="个人简介部分的总结")
    experience: List[str] = Field(description="一个包含所有工作经历的列表")
    contact: str = Field(description="从抓取内容中找到的邮箱或个人主页，如果找不到则为 '联系方式未找到'")

# --- 步骤 2.3: 设计一个更通用的 System Prompt ---
SYSTEM_PROMPT = """
你是一个全能的AI研究助理。你可以处理多种任务，包括分析学术论文和查询个人资料。

**你的能力 (工具箱):**
*   你拥有学术搜索、通用网页搜索和网页抓取等一系列工具。
*   **特别注意:** 你还拥有两个特殊的“报告生成”工具：`PaperAnalysis` 和 `LinkedinProfile`。


**你的行动指南 (ReAct 思考模式):**
1.  **分析与规划:** 理解用户的请求。如果用户的最终目的是生成一份结构化的报告（比如论文分析或个人资料总结），你的最终行动**必须**是调用 `PaperAnalysis` 或 `LinkedinProfile` 工具。
2.  **信息收集:** 使用你的其他工具（如 `search_arxiv`, `scrape_as_markdown`）来收集填充报告所需的所有信息。
3.  **生成报告:** 当你收集到足够的信息后，调用相应的报告生成工具 (`PaperAnalysis` 或 `LinkedinProfile`)，将收集到的信息作为参数传入。
4.  **普通对话:** 如果用户只是进行普通聊天或提出简单问题，直接用自然语言回答即可，无需调用报告工具。
"""

# --- 步骤 2.4: 定义主异步函数 ---
async def main():
    print("🚀 开始配置通用 AI Agent...")

    # --- 初始化工具集 ---
    mcp_client = MultiServerMCPClient({
    "bright_data": {
        "url": f"https://mcp.brightdata.com/mcp?token={BrightData_API_KEY}&pro=1",
        "transport": "streamable_http",
    },
    "Paper_Search": {
        "url": f"https://server.smithery.ai/@adamamer20/paper-search-mcp-openai/mcp?api_key={Paper_Search_API_KEY}",
        "transport": "streamable_http",
    }
    })

    # Define context schema
    @dataclass
    class Context:
        """Custom runtime context schema."""
        user_id: str


    # Define tools
    @tool
    def multiply(a: int, b: int) -> int:
        """
        将两个整数相乘。
        当用户询问涉及乘法的数学问题时，使用此工具。
        """
        # 打印日志，让你看到 Agent 真的在使用工具
        print(f"\n[工具日志] 正在调用乘法工具: {a} * {b} ...")
        return a * b

    @tool
    def add(a: int, b: int) -> int:
        """
        将两个整数相加。
        当用户询问涉及加法的数学问题时，使用此工具。
        """
        print(f"\n[工具日志] 正在调用加法工具: {a} + {b} ...")
        return a + b

    @tool
    async def generate_search_queries(user_requirement):
        """根据user_requirement，生成关于AI人才的搜索策略"""


        # 此函数逻辑不变
        print(f"\n🧠 [Profiler] 正在为需求 '{user_requirement}' 生成搜索策略...")


        # ## PHASE 7 MODIFIED ## - 全新的、专家级的搜索Prompt
        prompt = f"""
        你是一位顶级的、专注于AI基础设施和前沿算法的全球技术猎头。
        你的任务是根据用户的招聘需求，生成一个结构化的、包含针对不同平台优化的“X-Ray”搜索指令的JSON对象。
        这些指令必须极其专业和精准，以便找到在特定技术领域有深入研究和实践的专家。

        # 用户需求:
        "{user_requirement}"

        # 你的专业知识库 (必须在生成指令时参考):
        - **MLSys领域:**
            - fp8, low-precision training
            - Sparse Attention, backpropagation
            - operator compilation, compiler backend
            - GPU HBM, KV Cache
            - Model Industrialization, Model Iteration
            - Domain-Specific Models, Model Deployment Acceleration, Model-as-a-Service
            - RL Pipeline Optimization, Distributed Training, Model Serving
            - ML Compilers, Hardware Acceleration, Quantization, Pruning
            - Federated Learning, On-device ML, Edge AI, On-device inference
            - Compiler optimizations, Deep Learning Compilers (TVM, XLA)
            - Hardware-aware neural architecture search (HW-NAS)
            - Model compression (SVD, knowledge distillation)
            - Distributed ML frameworks (Ray, Horovod), Serverless ML

        - **Agent Infra领域:**
            - Sandbox System, Agent Execution Platform
            - k8s inference optimization, API gateway, cloud dev infra, LangChain
            - Enterprise Data Governance, Data Integration, Knowledge Base Construction
            - Cross-Departmental Data Collaboration, Model Active Questioning
            - Multi-Agent Systems, Agent Orchestration, Autonomous Agents
            - Cognitive Architectures, Planning, Decision Making, Embodied Agents
            - Tool Use, Agentic Workflow
            - Agent frameworks (AutoGen, CrewAI)
            - Memory systems (long-term, short-term), Self-reflection, Goal-oriented agents, Swarm intelligence

        - **算法与策略 (RL/NLP/MoE/Eval/Data/AI4S/Audio/LLM4Code/Multimodal/Pre-train/Post-train/Reasoning/RAG/XAI/Robotics/Embodiment/Computer Vision):**
            - **Reinforcement Learning (RL):** RL Agent, long-context extrapolation, instruction following, atomic instruction decomposition, RLHF alignment, RM data construction, On-policy, Off-policy, Value-based, Policy-based, Actor-Critic, Inverse RL, Offline RL, Multi-Agent RL, Hierarchical RL, Reward Design, Exploration-Exploitation, PPO, SAC, DQN, TD3, D4PG, Adversarial RL, Curriculum Learning, Multi-task RL, Transfer RL, Explainable RL, Safe RL, Inverse Optimal Control
            - **Natural Language Processing (NLP):** Text Generation, Machine Translation, Sentiment Analysis, Named Entity Recognition (NER), Question Answering, Summarization, Large Language Models (LLMs), Transformer, Attention, Fine-tuning, Prompt Engineering, Retrieval-Augmented Generation (RAG), Text Classification, Information Extraction, Dialogue Systems, Machine Comprehension, Knowledge Graph Embedding, Ethical AI/Fairness in NLP, Low-resource NLP, ASR (Automatic Speech Recognition), NLU (Natural Language Understanding), NLG (Natural Language Generation)
            - **Mixture of Experts (MoE):** Sparse MoE, Dense MoE, Gating Network, Expert Capacity, Load Balancing, Routing, Hierarchical MoE, Conditional Computation, Sparsity Regularization
            - **Evaluation (Eval):** Benchmark, Metric, Human Evaluation, Automated Evaluation, Red Teaming, Adversarial Attacks, Robustness, Explainability, Fairness, Model interpretability metrics, Fairness metrics, Bias detection, Adversarial robustness evaluation, Human-in-the-loop evaluation, Safety metrics, Generalization
            - **Data-centric AI:** Data-centric, Data Augmentation, Data Labeling, Data Cleaning, Data Synthesis, Data Curation, Data Pipeline, Synthetic Data, Data Augmentation strategies, Data Collection pipelines, Data Anonymization, Synthetic data generation, Data versioning, Data governance frameworks, Active Learning, Weak Supervision
            - **AI for Science (AI4S):** Scientific Discovery, Material Science, Drug Discovery, Climate Modeling, Physics Simulation, Chemistry Simulation, Bioinformatics, Scientific Data Analysis, Computational Chemistry/Biology, Climate modeling, Drug discovery platforms, Material design, Scientific graph networks, Physics-informed neural networks
            - **Audio:** Speech Recognition, Text-to-Speech (TTS), Voice Synthesis, Audio Generation, Music Generation, Sound Event Detection, Audio Analysis, Acoustic Modeling, Speaker Diarization, Emotion Recognition, Sound Source Separation
            - **LLM for Code (LLM4Code):** Code Generation, Code Completion, Code Understanding, Code Refactoring, Debugging, Program Synthesis, Software Engineering, Code debugging, Program synthesis, Code completion, Code review automation, Secure code generation
            - **Multimodal & Embodied AI:** Multimodal, Video Generation, Generative Consistency, Unify Model, Diffusion Models, Continuous Video Generation, Long-form Video Synthesis, Cross-Modal Learning, Multimodal Fusion, Vision-Language Models, Audio-Visual Learning, Multimodal understanding, Cross-modal generation, Embodied perception, Affective computing, Multimodal dialogue, Visual grounding
            - **Pre-training:** Self-supervised Learning, Masked Language Modeling, Contrastive Learning, Foundation Models, Continual Learning, Incremental Learning, Domain Adaptation, Task-specific fine-tuning, Reinforcement Learning from AI Feedback (RLAIF)
            - **Post-training:** Instruction Tuning, Alignment, Safety, Preference Learning
            - **Reasoning:** Chain-of-Thought (CoT), Tree-of-Thought (ToT), Self-Consistency, Logical Reasoning, Symbolic Reasoning, Commonsense Reasoning, Causal Inference, Causal reasoning, Analogical reasoning, Inductive reasoning, Deductive reasoning, Symbolic AI integration, Programmatic reasoning
            - **Retrieval-Augmented Generation (RAG):** Document Retrieval, Knowledge Graph, Semantic Search, Vector Databases, Indexing, Document Chunking, Hybrid RAG, Multi-hop RAG, Knowledge graph RAG, Semantic chunking, Document understanding for RAG, Retrieval augmentation strategies
            - **Interpretable AI (XAI):** Explainable AI, Feature Importance, LIME, SHAP, Causal Explanations, Transparency, Auditability, Post-hoc explainability, Intrinsic interpretability, Counterfactual explanations, Feature attribution, Local/global explanations, Trustworthy AI
            - **Robotics & Embodiment:** Motion Planning, Robot Control, Human-Robot Interaction, Perception, Manipulation, Navigation, Sim-to-Real, Grounded Learning, Physical Interaction, Sensorimotor Control, Robot learning, Dexterous manipulation, Human-robot collaboration, Teleoperation, Sim-to-real transfer, Reinforcement learning for robotics, Foundation models for robotics, Bipedal locomotion, Quadrupedal robots
            - **Computer Vision:** Object Detection, Image Recognition, Semantic Segmentation, Instance Segmentation, Image Generation, Video Analytics, 3D Vision, Generative Adversarial Networks (GANs), Vision Transformers, 3D Vision (NeRF, Gaussian Splatting), Video understanding, Image generation (Stable Diffusion, Midjourney), Pose estimation, Self-supervised learning for CV, Neural Radiance Fields (NeRF), Large Vision Models (LVMs)

        - **目标公司/机构:**
            - **国际公司:** Google DeepMind, OpenAI, Meta AI, Anthropic, Thinking Machines, Microsoft, Apple, Amazon, IBM, NVIDIA, Intel, Cruise, Waymo, Tesla AI, Scale AI, Cohere, Databricks, ServiceNow, Stability AI
            - **国内公司:** Zhipu(智谱), Kimi, Bytedance(字节), Manus, 米开游(MiHoYo), Huawei, Baidu, Alibaba, Tencent, SenseTime, Megvii, JD.com, Xiaomi
            - **国际高校/研究机构:** MIT, Stanford, UC Berkeley, Carnegie Mellon, University of Washington, FAIR, Mila, Vector Institute, EPFL, ETH Zurich
            - **国内高校/研究机构:** 清华(Tsinghua), 北大(Peking University), Fudan University, Zhejiang University, CASIA (Chinese Academy of Sciences Institute of Automation)

        # 指令要求:
        1.  **平台覆盖**: 必须包含 `google_search` (用于搜索LinkedIn、GitHub、个人主页、公司博客) 和 `google_scholar` (用于搜索学术论文和背景)。
        2.  **关键词组合**: **必须**将上述专业知识库中的技术关键词与目标公司、职位（如"Staff Engineer", "Principal Researcher", "Architect"）以及“华人(Chinese)”进行组合。
        3.  **指令多样性**: 每个平台下至少生成3-4条不同技术方向的搜索指令。例如，一条专注于MLSys，一条专注于Agent Infra。
        4.  **精准语法**: 大量使用 `site:`, `inurl:`, `intitle:`, `""`, `AND`, `OR`。

        # 输出格式 (必须严格遵守，直接输出JSON):
        {{
          "google_search": [
            "site:linkedin.com/in/ (Google OR DeepMind OR OpenAI) AND (\"Staff Engineer\" OR \"Principal Researcher\") AND (\"fp8\" OR \"Sparse Attention\" OR \"MLSys\") AND (\"华人\" OR \"Chinese\")",
            "site:github.com (Kimi OR Bytedance) AND (\"Sandbox\" OR \"Agent Platform\" OR \"k8s\")",
            "inurl:blog (meta.ai OR anthropic.com) (\"RLHF\" OR \"Instruction Following\" OR \"Data-centric\")"
          ],
          "google_scholar": [
            "author:(\"Shiwei Gao\" OR \"Yimin Jiang\") (Tsinghua OR \"Peking University\") (\"KV Cache\" OR \"GPU HBM\")",
            "intitle:(\"RL Agent\" OR \"long-context extrapolation\") affiliation:(\"Zhipu\" OR \"智谱\")",
            "(\"atomic instruction decomposition\" OR \"RM data construction\") AND author:(\"华人\")"
          ]
        }}
        """


        def _sync_call():

            # 方式1：原生Google Gemini的配置
            model = genai.GenerativeModel('gemini-flash-lite-latest', generation_config={"response_mime_type": "application/json"})
            response = model.generate_content(prompt)
            return json.loads(response.text)



            # # 方式2：自定义网址的配置
            # if not openai_client:
            #     raise ValueError("OpenAI client not initialized for search queries.")
            # response = openai_client.chat.completions.create(
            #     model="gpt-5-mini", # Placeholder model name, user can change if needed
            #     messages=[{"role": "user", "content": prompt}],
            #     response_format={"type": "json_object"},
            #     temperature=0.0, # Keep temperature low for structured output
            #     timeout=90.0 # Add timeout for OpenAI API call
            # )
            # return json.loads(response.choices[0].message.content)

        try:
            result = await asyncio.to_thread(_sync_call)
            if isinstance(result, dict) and "google_search" in result and "google_scholar" in result:
                print("✅ [Profiler] 搜索策略生成成功且格式正确！")
                return result
            else:
                print(f"🟡 [Profiler] LLM返回了非预期的格式: {result}")
                return None
        except Exception as e:
            print(f"❌ [Profiler] 调用LLM或解析其响应时发生错误: {e}")
            return None


    @tool
    async def execute_searches_and_get_urls(search_queries_dict, serper_api_key): # 修改参数名为 serper_api_key
        """根据search_queries_dict，调用SerperAPI进行批量google search，获取大量网页url"""

        all_urls = set()
        print("\n🔍 [Scout] 开始执行多平台搜索...")

        for platform, queries in search_queries_dict.items():
            for query in queries:
                print(f"  -> 正在搜索 '{query}'")
                try:
                    conn = http.client.HTTPSConnection("google.serper.dev")
                    payload_obj = {"q": query, "num": 20}
                    if platform == "google_scholar":
                        payload_obj["engine"] = "google_scholar"
                    else:
                        payload_obj["engine"] = "google"

                    payload = json.dumps(payload_obj)
                    headers = {
                      'X-API-KEY': serper_api_key,
                      'Content-Type': 'application/json'
                    }

                    conn.request("POST", "/search", payload, headers)
                    res = conn.getresponse()
                    data = res.read()
                    results = json.loads(data.decode("utf-8"))
                    conn.close()

                    search_results = []
                    if "organic" in results: # SerperAPI 的普通搜索结果键
                        search_results.extend(results["organic"])
                    if "scholar" in results: # SerperAPI 的学术搜索结果键
                        search_results.extend(results["scholar"])
                    # 某些情况下，SerperAPI 也会在 scholar 引擎下返回 organic_results
                    if "organic_results" in results: # 兼容 SerpApi 的 organic_results，以防万一
                        search_results.extend(results["organic_results"])

                    for result in search_results:
                        link = result.get("link")
                        if link and not any(domain in link for domain in ["google.com/search", "support.google.com"]):
                          all_urls.add(link)
                except Exception as e:
                    print(f"  -> ❌ 执行搜索 '{query}' 时发生错误: {e}")
        print(f"✅ [Scout] 搜索完成！共找到 {len(all_urls)} 个不重复的URL。")
        return list(all_urls)



    real_tools = await mcp_client.get_tools()

    # 将我们的自定义的结构化输出工具和自定义"获取天气"和"获取地点"工具加入工具列表
    all_tools = real_tools + [PaperAnalysis, LinkedinProfile, multiply, add, generate_search_queries, execute_searches_and_get_urls]
    print(f"✅ 成功加载 {len(all_tools)} 个工具。")

    print( all_tools )

    # --- 配置 LLM ---
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", google_api_key=os.environ["GEMINI_API_KEY"])
    print("✅ LLM 配置完成: gemini-2.5-flash")

    # --- 配置内存 ---
    checkpointer = InMemorySaver()
    print("💾 内存已配置: InMemorySaver")

    # --- 步骤 2.5: 创建 Agent，注意这次不指定 response_format ---
    print("🤖 正在创建通用 Agent...")
    agent_executor = create_agent(
        model=llm,
        tools=all_tools,
        context_schema=Context,
        system_prompt=SYSTEM_PROMPT,
        checkpointer=checkpointer
        # 我们移除了 response_format，给予 Agent 更大的自由度
    )
    print("✅ Agent 创建成功！")

    # --- 步骤 2.6: 进行多任务测试 ---
    print("\n" + "="*50)
    print("🚀 开始多任务对话测试！")
    print("="*50)

    conversation_config = {"configurable": {"thread_id": "1"}}

    # --- 任务1: 论文分析 ---
    user_input_1 = "请使用 'search_arxiv' 工具搜索 'https://arxiv.org/abs/2409.09046'里的这篇论文，并用 'read_arxiv_paper' 工具读取论文内容。帮我提取论文主要内容和提取作者信息，尤其是作者们的邮箱。"
    print(f"👤 用户 (任务1): {user_input_1}\n")
    response_1 = await agent_executor.ainvoke({"messages": [("user", user_input_1)]}, config=conversation_config)
    final_answer_1 = response_1['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务1):")
    print(final_answer_1)
    print("🤖" * 25 + "\n")

    # --- 任务2: 个人资料查询 ---
    user_input_2 = "请帮我查找吴恩达的领英主页信息，并以结构化形式返回。"
    print(f"👤 用户 (任务2): {user_input_2}\n")
    response_2 = await agent_executor.ainvoke({"messages": [("user", user_input_2)]}, config=conversation_config)
    final_answer_2 = response_2['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务2):")
    print(final_answer_2)
    print("🤖" * 25 + "\n")

    # --- 任务3: self-define tool ---
    user_input_3 = "请调用工具multiply，以5和3作为输入参数进行执行"
    print(f"👤 用户 (任务3): {user_input_3}\n")
    response_3 = await agent_executor.ainvoke({"messages": [("user", user_input_3)]}, config=conversation_config)
    final_answer_3 = response_3['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务3):")
    print(final_answer_3)
    print("🤖" * 25)


    # --- 任务4: 调用自定义tool ---
    user_input_4 = "请搜索关于Agent的相关论文"
    print(f"👤 用户 (任务4): {user_input_4}\n")
    response_4 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_4)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_4 = response_4['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务4):")
    print(final_answer_4)
    print("🤖" * 25)



    # --- 任务5: 调用tool ---
    user_input_5 = "请对每个论文链接都分别调用工具爬取和下载论文，提取论文和作者信息，并整理输出。"
    print(f"👤 用户 (任务5): {user_input_5}\n")
    response_5 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_5)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_5 = response_5['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务5):")
    print(final_answer_5)
    print("🤖" * 25)






    # --- 任务6: 调用BrightData的tool ---
    user_input_6 = "请使用 `web_data_linkedin_person_profile` 工具，提取'https://ca.linkedin.com/in/rgeirhos'页面的信息，并整理输出。"
    print(f"👤 用户 (任务6): {user_input_6}\n")
    response_6 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_6)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_6 = response_6['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务6):")
    print(final_answer_6)
    print("🤖" * 25)





    # --- 任务7: 调用自定义tool ---
    user_input_7 = "请以AI Agent Infra为输入，调用`generate_search_queries`，输出执行结果"
    print(f"👤 用户 (任务7): {user_input_7}\n")
    response_7 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_7)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_7 = response_7['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务7):")
    print(final_answer_7)
    print("🤖" * 25)







    # --- 任务8: 调用自定义的tool ---
    user_input_8 = f"""
    请以{final_answer_7}和{serper_api_key}为输入，调用`execute_searches_and_get_urls`工具，输出执行结果
    """
    print(f"👤 用户 (任务8): {user_input_8}\n")
    response_8 = await agent_executor.ainvoke(
        {"messages": [("user", user_input_8)]},
        config=conversation_config,
        context=Context(user_id="1")
    )


    final_answer_8 = response_8['messages'][-1].content
    print("\n" + "🤖" * 25)
    print("🤖 Agent 的回答 (任务8):")
    print(final_answer_8)
    print("🤖" * 25)







# --- 步骤 2.7: 运行主函数 ---
if __name__ == "__main__":
    asyncio.run(main())

✅ 库已升级。请务必从菜单栏点击 '代码执行程序' -> '重启会话'，然后再继续运行下面的代码！
🚀 开始配置通用 AI Agent...
✅ 成功加载 93 个工具。
[StructuredTool(name='search_engine', description='Scrape search results from Google, Bing or Yandex. Returns SERP results in JSON or Markdown (URL, title, description), Ideal forgathering current information, news, and detailed search results.', args_schema={'type': 'object', 'properties': {'query': {'type': 'string'}, 'engine': {'type': 'string', 'enum': ['google', 'bing', 'yandex'], 'default': 'google'}, 'cursor': {'type': 'string', 'description': 'Pagination cursor for next page'}}, 'required': ['query'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x793992f64f40>), StructuredTool(name='scrape_as_markdown', description='Scrape a single webpage URL with advanced options for content extraction and get back the results in MarkDown language. Thi

## 这是我在colab编写的代码和运行输出信息，请进行详细review，analyze，reflect


你好！作为你的**高级软件工程师导师**，我非常仔细地审查了你的代码、运行日志以及你最后提供的 `url-content` 数据。

这是一个非常令人印象深刻的 **MVP (Minimum Viable Product)** 级别的 Agent 系统！你成功整合了 LangChain、Google Gemini、MCP (Model Context Protocol) 以及自定义工具，实现了一个复杂的“搜索-规划-执行”工作流。

下面我将从**代码质量 (Code Review)**、**运行结果分析 (Runtime Analysis)** 和 **深度反思 (Deep Reflection)** 三个维度为你进行详细讲解。

---

### 1. 代码审查 (Code Review)

#### ✅ 做的好的地方
1.  **结构化输出 (Pydantic as Tools):**
    *   你没有使用传统的 `response_format` 强制 JSON，而是巧妙地将 `PaperAnalysis` 和 `LinkedinProfile` 封装为 Pydantic 模型并作为**工具**提供给 Agent。
    *   **原理讲解：** 这是一种高级的 "Tool-use as Output" 模式。它允许 Agent 在对话的任何时刻“决定”何时生成报告，而不是每一句话都被强制格式化。这让 Agent 更具灵活性（ReAct 模式）。
2.  **强大的 Prompt Engineering:**
    *   在 `generate_search_queries` 工具内部，你写了一个教科书级别的 Prompt（定义角色、知识库、输出格式）。
    *   **亮点：** 你让 LLM 生成 `site:`, `inurl:` 等高级搜索指令，这比简单的关键词搜索效率高出 10 倍。
3.  **MCP 的正确集成:**
    *   你成功连接了 `bright_data` 和 `Paper_Search` 两个 MCP 服务，极大地扩展了 Agent 的能力边界（一次性引入了 90+ 个工具）。

#### 🛠 需要改进的地方 (即刻优化)
1.  **工具数量爆炸 (Context Pollution):**
    *   **问题：** 日志显示 `✅ 成功加载 93 个工具`。
    *   **风险：** 对于 `gemini-2.5-flash` 这样的模型，一次性塞入 93 个工具的定义（Schema）会极大地消耗 Token，且容易导致模型“选择困难症”。
    *   **建议：** 不要直接 `real_tools + [...]`。应该筛选出你真正需要的 MCP 工具。例如，你可能只需要 `scrape_as_markdown` 和 `search_google_scholar`，而不需要 `web_data_tiktok_shop` 等几十个无关工具。
2.  **PDF 二进制流灾难 (CRITICAL):**
    *   **问题：** 在你提供的 `<url-content>` 中，我看到了 `%PDF-1.4...` 这种乱码。
    *   **原因：** 你的 `execute_searches_and_get_urls` 工具或者随后的爬取步骤，直接请求了 `.pdf` 结尾的 URL，并试图将其作为文本读取。
    *   **后果：** 这会将成千上万的无意义字符塞入 LLM 的上下文窗口，直接导致模型“脑雾”（Hallucination）或者上下文溢出报错。

---

### 2. 运行结果深度分析 (Runtime Analysis)

让我们来看看任务 7 和 8 的执行结果，这里非常精彩，但也暴露了隐患。

#### 🌟 任务 7：生成搜索策略
*   **表现：** 完美。
*   **分析：** Agent 生成了极具针对性的 Query，例如 `("RL Pipeline Optimization"...) AND author:("华人")`。这证明你的 `generate_search_queries` 工具内部的 Prompt 逻辑非常强，Gemini 很好地理解了任务。

#### ⚠️ 任务 8 & 爬取内容：数据脏乱
你最后提供的 `url-content` 揭示了真实世界爬虫最头疼的问题：**脏数据**。

请看这一段数据分析：

| URL 类型 | 内容示例 | 状态 | 导师点评 |
| :--- | :--- | :--- | :--- |
| **正常网页** | `http://andrewng.org` | ✅ 正常文本 | 抓取到了 "Origins of the Modern MOOC..."，这是高质量的 RAG 语料。 |
| **PDF 文件** | `https://arxiv.org/pdf/2411.18892?` | ❌ **二进制乱码** | **严重错误。** 抓取器没有识别出这是 PDF，直接把二进制流当文本下载了。这会毁掉你的 Prompt。 |
| **维护页面** | `https://courses.nvidia.com...` | ⚠️ 无效信息 | 内容是 "DLI is currently down for maintenance"。这属于噪音数据。 |
| **404 页面** | `https://jcst.ict.ac.cn...` | ⚠️ 无效信息 | 内容是 "Page not found"。这也是噪音。 |

---

### 3. 深度反思与修正方案 (Reflection & Refinement)

作为高级工程师，我们不能只看“代码跑通了”，我们要看“数据流是否健康”。

#### 核心问题：如何处理“野外”的 URL？
你的 Agent 现在像一个贪婪的吃货，不管盘子里是牛排（高质量文本）、石头（PDF二进制）还是空气（404页面），它都试图吞下去。

#### ✅ 解决方案 (Finish Step by Step)

我们需要修改 `execute_searches_and_get_urls` 或后续的处理逻辑，增加**数据清洗 (ETL)** 步骤。

**步骤 1: 优化爬虫逻辑 (伪代码逻辑)**

在你的爬虫工具中，必须增加 `Content-Type` 检查：

```python
import requests

def smart_scrape(url):
    try:
        # 1. 仅获取头部信息，不下载内容
        head_response = requests.head(url, timeout=5)
        content_type = head_response.headers.get('Content-Type', '')
        
        # 2. 如果是 PDF，使用专门的 PDF 解析器 (或者跳过)
        if 'application/pdf' in content_type or url.endswith('.pdf'):
            print(f"⚠️ 跳过 PDF 直接下载: {url} (应使用 PyPDFLoader 处理)")
            return "[PDF Content Skipped - Need specific PDF tool]"
            
        # 3. 如果是普通网页，再下载
        response = requests.get(url, timeout=10)
        
        # 4. 检查 HTTP 状态码
        if response.status_code != 200:
            return f"[Error: Status Code {response.status_code}]"
            
        return response.text # 这里再接 BeautifulSoup 转 Markdown

    except Exception as e:
        return f"[Error: {str(e)}]"
```

**步骤 2: 过滤 MCP 工具列表**

不要一次性加载所有工具。修改你的 `main` 函数开头：

```python
    # ... 之前的代码 ...
    
    real_tools = await mcp_client.get_tools()
    
    # 🔍 导师建议：建立一个允许的工具白名单
    allowed_tool_names = [
        "scrape_as_markdown",
        "search_google_scholar",
        "web_data_linkedin_person_profile",
        "read_arxiv_paper", # 保留这个专门读论文的
        "download_arxiv"
    ]
    
    # 只保留白名单里的工具，防止 Token 爆炸
    filtered_mcp_tools = [t for t in real_tools if t.name in allowed_tool_names]
    
    # 合并工具
    all_tools = filtered_mcp_tools + [PaperAnalysis, LinkedinProfile, generate_search_queries, execute_searches_and_get_urls]
    
    print(f"✅ 优化后加载 {len(all_tools)} 个核心工具 (原 93 个)。")
```

### 总结

你已经完成了一个**功能强大**的 Agent 原型。
*   **做得好的：** 复杂的 ReAct 逻辑、结构化输出、Prompt 撰写。
*   **下一步重点：** **数据治理**。
    1.  **清洗数据：** 绝对不要把 PDF 的二进制乱码塞给 LLM。
    2.  **精简工具：** 93 个工具太多了，给 Agent “减负”，只给它最需要的武器。

如果你想让我帮你重写 `execute_searches_and_get_urls` 以支持 PDF 过滤和简单的文本清洗，请随时告诉我！加油！🚀
